<a href="https://colab.research.google.com/github/hdpark1208/ASDChatbot/blob/ASDC/ASD_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit ipywidgets

     |████████████████████████████████| 6.1 MB 8.4 MB/s 
     |████████████████████████████████| 18.0 MB 128 kB/s 
     |████████████████████████████████| 234 kB 59.2 MB/s 
     |████████████████████████████████| 207 kB 71.4 MB/s 
     |████████████████████████████████| 2.1 MB 59.0 MB/s 
     |████████████████████████████████| 199 kB 54.6 MB/s 
     |████████████████████████████████| 635 kB 62.5 MB/s 
     |████████████████████████████████| 1.4 MB 56.9 MB/s 
     |████████████████████████████████| 68 kB 8.8 MB/s 
     |████████████████████████████████| 33.9 MB 18 kB/s 
     |████████████████████████████████| 49 kB 8.2 MB/s 
     |████████████████████████████████| 943 kB 57.5 MB/s 
     |████████████████████████████████| 3.2 MB 55.0 MB/s 
     |████████████████████████████████| 6.3 MB 55.0 MB/s 
  Created wheel for qiskit: filename=qiskit-0.28.0-py3-none-any.whl size=11090 sha256=0a46c3f7e7968a90802776169533823fcc99c0cda6e50e14bad160ad43d6b193
  Stored in directory: /root/.cache/pip/whe

In [3]:
!pip install pytorch-transformers

     |████████████████████████████████| 176 kB 9.0 MB/s 
     |████████████████████████████████| 1.2 MB 49.4 MB/s 
     |████████████████████████████████| 131 kB 63.7 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 7.7 MB 52.6 MB/s 
     |████████████████████████████████| 138 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

In [5]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [6]:
df = pd.read_table('/content/drive/MyDrive/Colab Notebooks/ASD/dataset.txt',header=None, error_bad_lines=False)

b'Skipping line 3173: expected 1 fields, saw 2\nSkipping line 3429: expected 1 fields, saw 2\n'


In [7]:
df[['sentence','label']] = pd.DataFrame(df[0].str.split('|', 1).tolist())

In [8]:
df = df.drop([0],axis=1)
df

,sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5817,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5818,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5819,너가 한 말 중에,0
5820,제갈대중 ㅇㅂ,0


In [9]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=777)
print(len(train_data),len(test_data))

4075 1747


In [21]:
train_data.iloc[0,1]

'0'

In [16]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [17]:
nsmc_train_dataset = NsmcDataset(train_data)
train_loader = DataLoader(nsmc_train_dataset, batch_size=4, shuffle=True, num_workers=2)

In [18]:
# 런타임 설정 바꿔줘야함
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device) 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [27]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

AttributeError: ignored